In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

#AZURE OPEN AI MODEL INTEGRATION
from langchain.chat_models import init_chat_model

#OpenAI API Key unavailable, but Azure OpenAI keys available. Hence, using Azure OpenAI model.
model = init_chat_model(
    os.getenv("AZURE_OPENAI_LLM_MODEL"),
    model_provider="azure_openai",
    deployment_name=os.getenv("AZURE_OPENAI_LLM_MODEL"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2025-01-01-preview"),
)


In [14]:
#Structred Output - Model can be requested to provide output in structured format matching a specific schema.

#Pydantic - Models to define structured output schema

from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The release year of the movie")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The IMDb rating of the movie")

In [15]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=AzureChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x71255971b490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x71255971bcd0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x71255974ed50>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x712559718c90>, model_name='gpt-5-chat', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://pfs-2-namit-resource.cognitiveservices.azure.com/', deployment_name='gpt-5-chat', openai_api_version='2025-01-01-preview', openai_api_type='azure'), kwargs={'response_format': <class '__main__.Movie'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title

In [16]:
response = model_with_structure.invoke("Provide details about the movie Inception.")
print(response)  # Structured output as per the Movie schema

title='Inception' year=2010 director='Christopher Nolan' rating=8.8


In [17]:
#Message output alongside parsed structured output

class Movie(BaseModel):
    title: str = Field(...,description="The title of the movie")
    year: int = Field(...,description="The release year of the movie")
    director: str = Field(...,description="The director of the movie")
    rating: float = Field(...,description="The IMDb rating of the movie")

model_with_structure = model.with_structured_output(Movie, include_raw = True)

response = model_with_structure.invoke("Provide details about the movie Inception.")
print(response)  # Structured output along with raw message content

{'raw': AIMessage(content='{"title":"Inception","year":2010,"director":"Christopher Nolan","rating":8.8}', additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 133, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CxWNWLZylkXeqfrcHEkspb3Vd8ny2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered':

In [19]:
#Nester Structured Output

class Actor(BaseModel):
    name: str = Field(...,description="The name of the actor")
    birth_year: int = Field(...,description="The birth year of the actor")

class MovieDetails(BaseModel):
    title: str = Field(...,description="The title of the movie")
    year: int = Field(...,description="The release year of the movie")
    cast: list[Actor] = Field(...,description="List of main actors in the movie")
    genre: str = Field(...,description="The genre of the movie")
    budget: int = Field(...,description="The budget of the movie in USD")


model_with_nested_structure = model.with_structured_output(MovieDetails, include_raw = True)

response = model_with_nested_structure.invoke("Provide detailed information about the movie The Dark Knight.")
print(response)  # Nested structured output along with raw message content

{'raw': AIMessage(content='{"title":"The Dark Knight","year":2008,"cast":[{"name":"Christian Bale","birth_year":1974},{"name":"Heath Ledger","birth_year":1979},{"name":"Aaron Eckhart","birth_year":1968},{"name":"Michael Caine","birth_year":1933},{"name":"Gary Oldman","birth_year":1958},{"name":"Maggie Gyllenhaal","birth_year":1977},{"name":"Morgan Freeman","birth_year":1937}],"genre":"Action, Crime, Drama","budget":185000000}', additional_kwargs={'parsed': MovieDetails(title='The Dark Knight', year=2008, cast=[Actor(name='Christian Bale', birth_year=1974), Actor(name='Heath Ledger', birth_year=1979), Actor(name='Aaron Eckhart', birth_year=1968), Actor(name='Michael Caine', birth_year=1933), Actor(name='Gary Oldman', birth_year=1958), Actor(name='Maggie Gyllenhaal', birth_year=1977), Actor(name='Morgan Freeman', birth_year=1937)], genre='Action, Crime, Drama', budget=185000000), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 229, 'total_t

In [20]:
# TypedDict - Simple alternative when runtime validation is not required

from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, "The title of the movie"]
    year: Annotated[int, "The release year of the movie"]
    director: Annotated[str, "The director of the movie"]
    rating: Annotated[float, "The IMDb rating of the movie"]

model_with_typedict = model.with_structured_output(MovieDict)

response = model_with_typedict.invoke("Provide details about the movie Interstellar.")
print(response)  # Structured output as per the MovieDict schema

{'title': 'Interstellar', 'year': 2014, 'director': 'Christopher Nolan', 'rating': 8.6}


In [21]:
#Nested TypedDict
class ActorDict(TypedDict):
    """An actor with details."""
    name: Annotated[str, "The name of the actor"]
    birth_year: Annotated[int, "The birth year of the actor"]

class MovieDetailsDict(TypedDict):
    """A movie with detailed information."""
    title: Annotated[str, "The title of the movie"]
    year: Annotated[int, "The release year of the movie"]
    cast: Annotated[list[ActorDict], "List of main actors in the movie"]
    genre: Annotated[str, "The genre of the movie"]
    budget: Annotated[int, "The budget of the movie in USD"]

model_with_nested_typedict = model.with_structured_output(MovieDetailsDict)

response = model_with_nested_typedict.invoke("Provide detailed information about the movie The Dark Knight.")
print(response)  # Nested structured output using TypedDict

{'title': 'The Dark Knight', 'year': 2008, 'cast': [{'name': 'Christian Bale', 'birth_year': 1974}, {'name': 'Heath Ledger', 'birth_year': 1979}, {'name': 'Aaron Eckhart', 'birth_year': 1968}, {'name': 'Michael Caine', 'birth_year': 1933}, {'name': 'Maggie Gyllenhaal', 'birth_year': 1977}, {'name': 'Gary Oldman', 'birth_year': 1958}, {'name': 'Morgan Freeman', 'birth_year': 1937}], 'genre': 'Action, Crime, Drama', 'budget': 185000000}


In [22]:
model_with_nested_typedict.profile

AttributeError: 'RunnableSequence' object has no attribute 'profile'

In [24]:
## Data Classes

## using Pydantic BaseModel for structured output
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str = Field(...,description="Full name of the contact")
    email: str = Field(...,description="Email address of the contact")
    phone: str = Field(...,description="Phone number of the contact")

agent = create_agent(model, response_format = ContactInfo)

result = agent.invoke({
    "messages":[
    {
        "role":"user",
        "content":"Extract contact information from the following text: 'John Doe can be reached at +1-234-567-8901 or john.doe@example.com "
    }
    ]
})

print(result["structured_response"])

name='John Doe' email='john.doe@example.com' phone='+1-234-567-8901'


In [26]:
## compare with Data Classes

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfoDataClass:
    name: str
    email: str
    phone: str

agent_dataclass = create_agent(model, response_format = ContactInfoDataClass)

result_dataclass = agent_dataclass.invoke({
    "messages":[
    {
        "role":"user",
        "content":"Extract contact information from the following text: 'John Doe can be reached at +1-234-567-8901 or john.doe@example.com "
    }
    ]
})

print(result_dataclass["structured_response"])

ContactInfoDataClass(name='John Doe', email='john.doe@example.com', phone='+1-234-567-8901')
